In [1]:
from __future__ import annotations # PEP 563: Postponed Evaluation of Annotations
from dataclasses import dataclass # https://docs.python.org/3/library/dataclasses.html I like these a lot

@dataclass
class Curve:
    """
    Elliptic Curve over the field of integers modulo a prime.
    Points on the curve satisfy y^2 = x^3 + a*x + b (mod p).
    """
    p: int # the prime modulus of the finite field
    a: int
    b: int

In [2]:
@dataclass
class Point:
    """ An integer point (x,y) on a Curve """
    curve: Curve
    x: int
    y: int

In [3]:
INF = Point(None, None, None) # special point at "infinity", kind of like a zero

def extended_euclidean_algorithm(a, b):
    """
    Returns (gcd, x, y) s.t. a * x + b * y == gcd
    This function implements the extended Euclidean
    algorithm and runs in O(log b) in the worst case,
    taken from Wikipedia.
    """
    old_r, r = a, b
    old_s, s = 1, 0
    old_t, t = 0, 1
    while r != 0:
        quotient = old_r // r
        old_r, r = r, old_r - quotient * r
        old_s, s = s, old_s - quotient * s
        old_t, t = t, old_t - quotient * t
    return old_r, old_s, old_t

def inv(n, p):
    """ returns modular multiplicate inverse m s.t. (n * m) % p == 1 """
    gcd, x, y = extended_euclidean_algorithm(n, p) # pylint: disable=unused-variable
    return x % p

def elliptic_curve_addition(self, other: Point) -> Point:
    # handle special case of P + 0 = 0 + P = 0
    if self == INF:
        return other
    if other == INF:
        return self
    # handle special case of P + (-P) = 0
    if self.x == other.x and self.y != other.y:
        return INF
    # compute the "slope"
    if self.x == other.x: # (self.y = other.y is guaranteed too per above check)
        m = (3 * self.x**2 + self.curve.a) * inv(2 * self.y, self.curve.p)
    else:
        m = (self.y - other.y) * inv(self.x - other.x, self.curve.p)
    # compute the new point
    rx = (m**2 - self.x - other.x) % self.curve.p
    ry = (-(m*(rx - self.x) + self.y)) % self.curve.p
    return Point(self.curve, rx, ry)

Point.__add__ = elliptic_curve_addition # monkey patch addition into the Point class

In [4]:
def double_and_add(self, k: int) -> Point:
    assert isinstance(k, int) and k >= 0
    result = INF
    append = self
    while k:
        if k & 1:
            result += append  #addition
        append += append  #2P  double 
        k >>= 1
    return result

# monkey patch double and add into the Point class for convenience
Point.__rmul__ = double_and_add

In [5]:

def elliptic_curve_subtraction(self, other: Point) -> Point:
    result = Point(test_curve,    x = 0x0,    y = 0x0)
    self.y = self.y *(-1)
    result= elliptic_curve_addition(self,other)
    result.y = result.y * (-1) %test_curve.p
    return result

Point.__sub__ = elliptic_curve_subtraction

In [6]:
test_curve = Curve(
    p = 0x43,
    a = 0x2, # a = 2
    b = 0x3, # b = 3
)

test_G = Point(
    test_curve,
    x = 0x2,
    y = 0x16,
)

# we can verify that the generator point is indeed on the curve, i.e. y^2 = x^3 + 2x+ 3 (mod p)
print("Generator IS on the curve: ", (test_G.y**2 - test_G.x**3 -test_curve.a*test_G.x - test_curve.b) % test_curve.p == 0)

test_secret_key = 4

test_public_key = test_secret_key * test_G

print(f"x: {test_public_key.x}\ny: {test_public_key.y}")
print("Verify the public key is on the curve: ", (test_public_key.y**2 - test_public_key.x**3 - test_curve.a*test_public_key.x - test_curve.b - 67) % test_curve.p == 0)

r = 2

M=Point(test_curve, x=0x18,y=0x1a,)
print("M IS on the curve: ", (M.y**2 - M.x**3 -test_curve.a*M.x - test_curve.b - 67) % test_curve.p == 0)

C1 = r * test_G
C2 = M + r*test_public_key
print("C1 is: ", C1, "C2 is: ", C2 )


print("decryption: ", C2-test_secret_key*C1)


Generator IS on the curve:  True
x: 13
y: 45
Verify the public key is on the curve:  True
M IS on the curve:  True
C1 is:  Point(curve=Curve(p=67, a=2, b=3), x=35, y=1) C2 is:  Point(curve=Curve(p=67, a=2, b=3), x=21, y=44)
decryption:  Point(curve=Curve(p=67, a=2, b=3), x=24, y=26)
